In [3]:
import pandas as pd


In [5]:
excel_file_path = 'Qualifizierungsmodule_Qualifizierungspläne_v4 (1).xlsx'

In [26]:
df = pd.read_excel(excel_file_path, sheet_name='Tabelle1')


In [27]:
# Drop the first two rows
df = df.drop([0, 1]).reset_index(drop=True)

# Drop the first column
df = df.drop(df.columns[0], axis=1)

In [28]:
df.columns = df.iloc[0]

In [29]:
df = df.drop(0).reset_index(drop=True)


In [32]:
df.columns.values[0] = "Kompetenz"
df.columns.values[1] = "Definition"

In [34]:
df

,Kompetenz,Definition,Kennen,Verstehen,Anwenden,Beherrschen
0,Systemisches Denken,Die Kompetenz fundamentale Konzepte des System...,Der Teilnehmer kennt die Zusammenhänge seines ...,Der Teilnehmer versteht die Wechselwirkung der...,"Der Teilnehmer ist in der Lage, sein vorliegen...",Der Teilnehmer ist in der Lage Systemisches De...
1,Systemmodellierung und -analyse,Die Kompetenz exakte Daten und Informationen m...,Die Teilnehmer kennen die Grundlage der Modell...,Die Teilnehmer verstehen wie Modelle ihre Arbe...,Die Teilnehmer sind in der Lage eigene Systemm...,Die Teilnehmer können Vorgaben für (notwendige...
2,Berücksichtigung von Systemlebenszyklusphasen,Die Kompetenz während der Realisierung eines S...,Die Teilnehmer kennt die Lebenszyklusphasen se...,Der Teilnehmer verseht warum und wie alle Lebe...,Der Teilmnehmer ist in der Lage für die für ih...,Der Teilnehmer ist in der Lage Konzepte hinsic...
3,Agiles Denken / Kunden-Nutzenorientierung,Die Kompetenz die agilen Werte / den Kundennut...,Die Teilnehmerin ikennt die Grundprinzipien de...,Die Teilnehmerin versteht wie Agiles Denken in...,Die Teilnehmerin ist ind er Lage ein System na...,Die Teilnehmerin ist in der Lage Agilkes Denke...
4,Anforderungsmanagement,Die Kompetenz Bedürfnisse und Erwartungen der...,"Prozess: Die Teilnehmer können Bedarfe, Stake...",Prozess: Die Teilnehmer verstehen wie Anforder...,Prozess: Der Teilnehmer kann selbständig Anfor...,Prozess: Die Teilnehmer sind in der Lage Unzul...
5,System-Architekturgestaltung,Die Kompetenz die zu einem System zugehörigen ...,Prozess: Die Teilnehmer kennen den Zweck von A...,"Prozess: Die Teilnehmer können nachvollziehen,...",Prozess: Die Teilnehmer kennen relevante Proze...,Prozess: Die Teilnehmer sind in der Lage Unzul...
6,"Integration, Verifikation & Validierung",Die Kompetenz eine Menge von Systemelementen z...,Der Teilnehmer kennt die Ziele der Verifikatio...,"Der Teilnehmer kann Testpläne, Testfälle und -...",Der Teilnehmer kann Testpläne erstellen. Er is...,Der Teilnehmer ist in der Lage selbständig und...
7,"Betrieb, Service und Instandhaltung","Die Kompetenz das System in Betrieb zu nehmen,...","Die Teilnehmer kennen die Zeile der Betriebs-,...","Die Teilnehmer verstehen, wie die Phasen Betri...","Die Teilnehmer verstehen, wie die Phasen Betri...",Die Teilnehmer sind in der Lage die Organisati...
8,Agile Methodenkompetenz,"Die Kompetenz Methoden anzuwenden, welche die ...",Methode: Der Teilnehmer kennt die Agilen Werte...,Methode: Der Teilnehmer versteht die Grundlage...,Methode: der Teilnehmer kann in einem agilen U...,Der Teilnehmer kann für ein projekt die releva...
9,Selbstorganisation,Die Kompetenz sich selbst zu organisieren und ...,Die Teilnehmer kennen die Konzepte der Selbsto...,Die Teilnehmer verstrehen wie die Konzepte der...,Die Teilnehmer sind in der Lage selbstorganisi...,Die Teilnehmer sind in der Lage selbstorganisi...


## Generate indicators using LLM 

In [35]:
import os
from pydantic import BaseModel, Field
from typing import List, Dict
from langchain.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI

In [36]:
# --- Define the Pydantic model for competency indicators ---
class Indicator(BaseModel):
    english: str = Field(..., description="The indicator in English.")
    german: str = Field(..., description="The indicator in German.")

class CompetencyLevelIndicators(BaseModel):
    kennen: List[Indicator] = Field(..., description="List of indicators for level 'Kennen'.")
    verstehen: List[Indicator] = Field(..., description="List of indicators for level 'Verstehen'.")
    anwenden: List[Indicator] = Field(..., description="List of indicators for level 'Anwenden'.")
    beherrschen: List[Indicator] = Field(..., description="List of indicators for level 'Beherrschen'.")

class CompetencyIndicatorsResponse(BaseModel):
    competency_name: str = Field(..., description="Name of the competency.")
    indicators: CompetencyLevelIndicators = Field(..., description="Structured indicators for each competency level.")

    # Method to return JSON output
    def to_json(self):
        return self.model_dump()  # Convert the Pydantic model to a JSON-friendly dictionary


In [ ]:
# Initialize the Azure OpenAI client
azure_endpoint = os.environ.get("AZURE_API_BASE")
api_key = os.environ.get("AZURE_API_KEY")

In [38]:
def init_llm():
    """
    Initialize the Azure OpenAI LLM.
    """
    return AzureChatOpenAI(
        model="gpt-4",
        openai_api_key=api_key,
        azure_endpoint=azure_endpoint,
        api_version="2024-02-01",
        temperature=0.7
    )

In [76]:

# --- Create the prompt for generating indicators ---
def create_competency_prompt():
    """
    Create a system and user prompt for generating competency indicators for each level in both English and German.
    """
    system_prompt = "You are an expert in systems engineering competency assessment."
    
    user_prompt = """
    I am creating a self-assessment survey for the competency "{competency_name}" with the following definition:
    
    "{competency_definition}"
    
    Please generate 1-2 user-friendly self-assessment indicators for each competency level ('Kennen', 'Verstehen', 'Anwenden', 'Beherrschen') in the format:
    
    'You are able to [do something]'.

    Each indicator should be provided in both English and German. Return the output in JSON format like this:
    {{
        "competency_name": "{competency_name}",
        "indicators": {{
            "kennen": [{{"english": "Indicator text", "german": "Indikator-Text"}}, ...],
            "verstehen": [{{"english": "Indicator text", "german": "Indikator-Text"}}, ...],
            "anwenden": [{{"english": "Indicator text", "german": "Indikator-Text"}}, ...],
            "beherrschen": [{{"english": "Indicator text", "german": "Indikator-Text"}}, ...]
        }}
    }}
    
    Here are the initial indicators for each level:
    {level_indicators}
    """

    return ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("user", user_prompt)
        ]
    )

In [77]:
# --- Create the competency chain ---
def create_competency_chain(llm,prompt):
    """
    Create the LangChain-style competency chain using the prompt and structured LLM.
    """
    prompt=prompt
    structured_llm = llm.with_structured_output(CompetencyIndicatorsResponse)
    return prompt | structured_llm

In [82]:
# --- Main function to generate indicators ---
def generate_competency_indicators(row):
    """
    Use the LLM to generate competency indicators and return the structured JSON.
    
    :param row: A row from the DataFrame containing competency information.
    :return: Competency indicators as structured JSON.
    """
    llm = init_llm()  # Initialize the LLM
    prompt = create_competency_prompt()  # Create the prompt
    competency_chain = create_competency_chain(llm,prompt)  # Create the competency chain
    #Prepare input to llm
    levels = ['Kennen', 'Verstehen', 'Anwenden', 'Beherrschen']
    level_indicators = "\n".join([f"{level.capitalize()} indicator: {row[level]}" for level in levels])

    inputs = {
        "competency_name": row['Kompetenz'],
        "competency_definition": row['Definition'],
        "level_indicators": level_indicators
    }
    # Run the competency chain
    result = competency_chain.invoke(inputs)
    
    # Return the structured response in JSON format
    return result.to_json()

In [58]:
df.columns = df.columns.str.strip()  # Strip any leading/trailing spaces


In [59]:
# Remove any potential index name from the columns
df.columns.name = None
# Process only the first row in the DataFrame for testing
first_row = df.iloc[0]  # Get the first row as a Series



In [60]:
print(first_row.index)  # This will show you the exact labels used in the Series


Index(['Kompetenz', 'Definition', 'Kennen', 'Verstehen', 'Anwenden',
       'Beherrschen'],
      dtype='object')


In [64]:
first_row['Kompetenz']

'Systemisches Denken'

In [68]:
competency_name = first_row['Kompetenz']
competency_definition = first_row['Definition']
levels = ['Kennen', 'Verstehen', 'Anwenden', 'Beherrschen']
system_prompt = "You are an expert in systems engineering competency assessment."
levels = ['Kennen', 'Verstehen', 'Anwenden', 'Beherrschen']
user_prompt = f"""
    I am creating a self-assessment survey for the competency "{competency_name}" with the following definition:
    
    "{competency_definition}"
    
    Please generate 1-2 user-friendly self-assessment indicators for each competency level ('Kennen', 'Verstehen', 'Anwenden', 'Beherrschen') in the format:
    
    'You are able to [do something]'.

    Each indicator should be provided in both English and German. Return the output in JSON format like this:
    {{
        "competency_name": "{competency_name}",
        "indicators": {{
            "kennen": [{{"english": "Indicator text", "german": "Indikator-Text"}}, ...],
            "verstehen": [{{"english": "Indicator text", "german": "Indikator-Text"}}, ...],
            "anwenden": [{{"english": "Indicator text", "german": "Indikator-Text"}}, ...],
            "beherrschen": [{{"english": "Indicator text", "german": "Indikator-Text"}}, ...]
        }}
    }}
    
    Here are the initial indicators for each level:
    """
    # Append each level indicator from the row
for level in levels:
    user_prompt += f"\n{level.capitalize()} indicator: {first_row[level]}"

In [70]:
print(user_prompt)


    I am creating a self-assessment survey for the competency "Systemisches Denken" with the following definition:
    
    "Die Kompetenz fundamentale Konzepte des Systemdenkens im Systems Engineering anzuwenden und die Rolle des eigenen Systems in seinem Gesamtkontext zu verstehen."
    
    Please generate 1-2 user-friendly self-assessment indicators for each competency level ('Kennen', 'Verstehen', 'Anwenden', 'Beherrschen') in the format:
    
    'You are able to [do something]'.

    Each indicator should be provided in both English and German. Return the output in JSON format like this:
    {
        "competency_name": "Systemisches Denken",
        "indicators": {
            "kennen": [{"english": "Indicator text", "german": "Indikator-Text"}, ...],
            "verstehen": [{"english": "Indicator text", "german": "Indikator-Text"}, ...],
            "anwenden": [{"english": "Indicator text", "german": "Indikator-Text"}, ...],
            "beherrschen": [{"english": "Indicat

In [83]:
competency_json = generate_competency_indicators(first_row)
print(f"\nCompetency 1 JSON Output:\n{competency_json}\n")


Competency 1 JSON Output:
{'competency_name': 'Systemisches Denken', 'indicators': {'kennen': [{'english': 'You are able to identify the relationships within your system and its boundaries.', 'german': 'Sie sind in der Lage, die Zusammenhänge Ihres Systems und die dazugehörigen Systemgrenzen zu erkennen.'}], 'verstehen': [{'english': 'You understand the interactions of the individual components that make up the system.', 'german': 'Sie verstehen die Wechselwirkungen der einzelnen Komponenten, die das System ausmachen.'}], 'anwenden': [{'english': 'You are able to analyze your existing system and derive continuous improvements from it.', 'german': 'Sie sind in der Lage, Ihr vorhandenes System zu analysieren und daraus kontinuierliche Verbesserungen abzuleiten.'}], 'beherrschen': [{'english': 'You are able to carry systemic thinking into the company and inspire others for it.', 'german': 'Sie sind in der Lage, systemisches Denken in das Unternehmen zu tragen und andere dafür zu begeiste

In [84]:
import psycopg2
import json

In [ ]:

conn = psycopg2.connect(
    host=os.environ.get("POSTGRES_HOST"),
    database=os.environ.get("POSTGRES_DB"),
    user=os.environ.get("POSTGRES_USER"),
    password=os.environ.get("POSTGRES_PASSWORD")
)
cur = conn.cursor()

In [86]:
# Step 3: Query the competency table to create a lookup for competency IDs
cur.execute("SELECT id, competency_name FROM public.competency;")
competency_id_lookup = {row[1]: row[0] for row in cur.fetchall()}

In [87]:
competency_id_lookup

{'Systems Thinking': 1,
 'Lifecycle Consideration': 4,
 'Customer / Value Orientation': 5,
 'Systems Modeling and Analysis': 6,
 'Communication': 7,
 'Leadership': 8,
 'Self-Organization': 9,
 'Project Management': 10,
 'Decision Management': 11,
 'Information Management': 12,
 'Configuration Management': 13,
 'Requirements Definition': 14,
 'System Architecting': 15,
 'Integration, Verification,  Validation': 16,
 'Operation and Support': 17,
 'Agile Methods': 18}

In [88]:
# Create a mapping of English competency names to German equivalents
competency_mapping = {
    "Systems Thinking": "Systemisches Denken",
    "Lifecycle Consideration": "Berücksichtigung von Systemlebenszyklusphasen",
    "Customer / Value Orientation": "Agiles Denken / Kunden-Nutzenorientierung",
    "Systems Modeling and Analysis": "Systemmodellierung und -analyse",
    "Communication": "Kommunikation & Zusammenarbeit",
    "Leadership": "Führen",
    "Self-Organization": "Selbstorganisation",
    "Project Management": "Projektmanagement",
    "Decision Management": "Entscheidungsmanagement",
    "Information Management": "Informationsmanagement",
    "Configuration Management": "Konfigurationsmanagement",
    "Requirements Definition": "Anforderungsmanagement",
    "System Architecting": "System-Architekturgestaltung",
    "Integration, Verification,  Validation": "Integration, Verifikation & Validierung",
    "Operation and Support": "Betrieb, Service und Instandhaltung",
    "Agile Methods": "Agile Methodenkompetenz",
    "Implementation": "Implementierung"
}


In [89]:
# Verify if all German names from LLM can be matched with English names in competency_id_lookup
print("\nVerifying if German names match English names:")
unmatched_german_names = []

for german_name in competency_mapping.values():
    matched_english_name = next((eng_name for eng_name, ger_name in competency_mapping.items() if ger_name == german_name), None)
    if matched_english_name and matched_english_name in competency_id_lookup:
        print(f"German Competency '{german_name}' successfully matched to English Competency '{matched_english_name}' with ID {competency_id_lookup[matched_english_name]}")
    else:
        unmatched_german_names.append(german_name)

# Print any unmatched German names
if unmatched_german_names:
    print("\nUnmatched German Competency Names:")
    for name in unmatched_german_names:
        print(f"'{name}'")
else:
    print("\nAll German competency names matched successfully.")


Verifying if German names match English names:
German Competency 'Systemisches Denken' successfully matched to English Competency 'Systems Thinking' with ID 1
German Competency 'Berücksichtigung von Systemlebenszyklusphasen' successfully matched to English Competency 'Lifecycle Consideration' with ID 4
German Competency 'Agiles Denken / Kunden-Nutzenorientierung' successfully matched to English Competency 'Customer / Value Orientation' with ID 5
German Competency 'Systemmodellierung und -analyse' successfully matched to English Competency 'Systems Modeling and Analysis' with ID 6
German Competency 'Kommunikation & Zusammenarbeit' successfully matched to English Competency 'Communication' with ID 7
German Competency 'Führen' successfully matched to English Competency 'Leadership' with ID 8
German Competency 'Selbstorganisation' successfully matched to English Competency 'Self-Organization' with ID 9
German Competency 'Projektmanagement' successfully matched to English Competency 'Proje

In [90]:
df

,Kompetenz,Definition,Kennen,Verstehen,Anwenden,Beherrschen
0,Systemisches Denken,Die Kompetenz fundamentale Konzepte des System...,Der Teilnehmer kennt die Zusammenhänge seines ...,Der Teilnehmer versteht die Wechselwirkung der...,"Der Teilnehmer ist in der Lage, sein vorliegen...",Der Teilnehmer ist in der Lage Systemisches De...
1,Systemmodellierung und -analyse,Die Kompetenz exakte Daten und Informationen m...,Die Teilnehmer kennen die Grundlage der Modell...,Die Teilnehmer verstehen wie Modelle ihre Arbe...,Die Teilnehmer sind in der Lage eigene Systemm...,Die Teilnehmer können Vorgaben für (notwendige...
2,Berücksichtigung von Systemlebenszyklusphasen,Die Kompetenz während der Realisierung eines S...,Die Teilnehmer kennt die Lebenszyklusphasen se...,Der Teilnehmer verseht warum und wie alle Lebe...,Der Teilmnehmer ist in der Lage für die für ih...,Der Teilnehmer ist in der Lage Konzepte hinsic...
3,Agiles Denken / Kunden-Nutzenorientierung,Die Kompetenz die agilen Werte / den Kundennut...,Die Teilnehmerin ikennt die Grundprinzipien de...,Die Teilnehmerin versteht wie Agiles Denken in...,Die Teilnehmerin ist ind er Lage ein System na...,Die Teilnehmerin ist in der Lage Agilkes Denke...
4,Anforderungsmanagement,Die Kompetenz Bedürfnisse und Erwartungen der...,"Prozess: Die Teilnehmer können Bedarfe, Stake...",Prozess: Die Teilnehmer verstehen wie Anforder...,Prozess: Der Teilnehmer kann selbständig Anfor...,Prozess: Die Teilnehmer sind in der Lage Unzul...
5,System-Architekturgestaltung,Die Kompetenz die zu einem System zugehörigen ...,Prozess: Die Teilnehmer kennen den Zweck von A...,"Prozess: Die Teilnehmer können nachvollziehen,...",Prozess: Die Teilnehmer kennen relevante Proze...,Prozess: Die Teilnehmer sind in der Lage Unzul...
6,"Integration, Verifikation & Validierung",Die Kompetenz eine Menge von Systemelementen z...,Der Teilnehmer kennt die Ziele der Verifikatio...,"Der Teilnehmer kann Testpläne, Testfälle und -...",Der Teilnehmer kann Testpläne erstellen. Er is...,Der Teilnehmer ist in der Lage selbständig und...
7,"Betrieb, Service und Instandhaltung","Die Kompetenz das System in Betrieb zu nehmen,...","Die Teilnehmer kennen die Zeile der Betriebs-,...","Die Teilnehmer verstehen, wie die Phasen Betri...","Die Teilnehmer verstehen, wie die Phasen Betri...",Die Teilnehmer sind in der Lage die Organisati...
8,Agile Methodenkompetenz,"Die Kompetenz Methoden anzuwenden, welche die ...",Methode: Der Teilnehmer kennt die Agilen Werte...,Methode: Der Teilnehmer versteht die Grundlage...,Methode: der Teilnehmer kann in einem agilen U...,Der Teilnehmer kann für ein projekt die releva...
9,Selbstorganisation,Die Kompetenz sich selbst zu organisieren und ...,Die Teilnehmer kennen die Konzepte der Selbsto...,Die Teilnehmer verstrehen wie die Konzepte der...,Die Teilnehmer sind in der Lage selbstorganisi...,Die Teilnehmer sind in der Lage selbstorganisi...


In [91]:
# Process each row in the DataFrame and get the structured JSON output
for index, row in df.iterrows():
    # Skip the iteration if the 'Kompetenz' column has the value 'Implementierung'
    if row['Kompetenz'] == 'Implementierung':
        print(f"Skipping row {index + 1} as it is 'Implementierung'")
        continue

    llm_output = generate_competency_indicators(row)
    print(f"\nCompetency {index + 1} JSON Output:\n{llm_output}\n")
    
    # Get the competency_id using the German name
    german_competency_name = llm_output['competency_name']
    english_competency_name = next((eng_name for eng_name, ger_name in competency_mapping.items() if ger_name == german_competency_name), None)
    competency_id = competency_id_lookup.get(english_competency_name)
    
    if competency_id:
        for level, indicators in llm_output['indicators'].items():
            for indicator in indicators:
                # Extract English and German indicators
                indicator_en = indicator['english']
                indicator_de = indicator['german']
                
                # Insert each indicator into the modified table
                cur.execute(
                    "INSERT INTO public.competency_indicators (competency_id, level, indicator_en, indicator_de) VALUES (%s, %s, %s, %s)",
                    (competency_id, level, indicator_en, indicator_de)
                )
        conn.commit()
        print(f"Inserted indicators for competency '{english_competency_name}'")
    else:
        print(f"Competency '{german_competency_name}' not found in mapping")



Competency 1 JSON Output:
{'competency_name': 'Systemisches Denken', 'indicators': {'kennen': [{'english': 'You are able to recognize the interrelationships of your system and its boundaries.', 'german': 'Sie kennen die Zusammenhänge Ihres Systems und die dazugehörigen Systemgrenzen.'}], 'verstehen': [{'english': 'You understand the interaction of the individual components that make up the system.', 'german': 'Sie verstehen die Wechselwirkung der einzelnen Komponenten, die das System ausmachen.'}], 'anwenden': [{'english': 'You are able to analyze your present system and derive continuous improvements from it.', 'german': 'Sie sind in der Lage, Ihr vorliegendes System zu analysieren und daraus kontinuierliche Verbesserungen abzuleiten.'}], 'beherrschen': [{'english': 'You are able to carry systemic thinking into the company and inspire others for it.', 'german': 'Sie sind in der Lage, systemisches Denken in das Unternehmen zu tragen und andere dafür zu begeistern.'}]}}

Inserted indic

In [ ]:
# Close the cursor and connection
cur.close()
conn.close()